In [5]:
from time import time
import timeit
import os
import numpy as np
import xarray as xr
from smmregrid import cdo_generate_weights, Regridder
from smmregrid.checker import check_cdo_regrid
from cdo import Cdo
cdo = Cdo()


In [6]:

indir='tests/data'
filelist = ['tas-ecearth.nc', '2t-era5.nc', 'tos-fesom.nc']
tfile = os.path.join(indir, 'r360x180.nc')
methods = ['nn','con']
for filein in filelist: 
    for method in methods :
        cc = check_cdo_regrid(os.path.join(indir,filein), tfile, method = method)
        print(filein + ': remap' + method + '->' + str(cc))


tas-ecearth.nc: remapnn->True
tas-ecearth.nc: remapcon->True
tas-ecearth.nc: remapbil->True
2t-era5.nc: remapnn->True
2t-era5.nc: remapcon->True
2t-era5.nc: remapbil->True
tos-fesom.nc: remapnn->True
tos-fesom.nc: remapcon->False


CalledProcessError: Command '['cdo', 'genbil,tests/data/r360x180.nc', 'tests/data/tos-fesom.nc', '/tmp/tmpavox5uxd']' returned non-zero exit status 1.

Test the smmregrid support for DataFrame instead of DataArray.

In [2]:
indir='tests/data'
filelist = ['tos-fesom.nc']
tfile = os.path.join(indir, 'r360x180.nc')

# interpolation with smmregrid
for filein in filelist: 
    xfield = xr.open_mfdataset(os.path.join(indir,filein))
    wfield = cdo_generate_weights(os.path.join(indir,filein), tfile, method = 'con')
    interpolator = Regridder(weights=wfield)
    var = list(xfield.data_vars)[-1]
    rfield = interpolator.regrid(xfield)

rfield



<xarray.Dataset>
Dimensions:  (time: 1, lat: 180, lon: 360)
Coordinates:
  * time     (time) datetime64[ns] 2011-01-16T12:00:00
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
Data variables:
    tos      (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>

In [3]:
# interpolation with smmregrid
for filein in filelist: 
    xfield = xr.open_mfdataset(os.path.join(indir,filein))
    cdofield = cdo.remapcon(tfile, input = os.path.join(indir,filein), returnXDataset = True)

cdofield

<xarray.Dataset>
Dimensions:    (time: 1, bnds: 2, lon: 360, lat: 180)
Coordinates:
  * time       (time) datetime64[ns] 2011-01-16T12:00:00
  * lon        (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    tos        (time, lat, lon) float32 ...
Attributes: (12/42)
    CDI:                    Climate Data Interface version 2.1.1 (https://mpi...
    Conventions:            CF-1.7 CMIP-6.2
    source:                 AWI-CM-1-1-MR
    institution:            Alfred Wegener Institute, Helmholtz Centre for Po...
    frequency:              mon
    activity_id:            CMIP
    ...                     ...
    parent_mip_era:         CMIP6
    parent_source_id:       AWI-CM-1-1-MR
    parent_time_units:      days since 2401-1-1
    parent_variant_label:   r1i1p1f1
    history:                Mon Jan 30 12:50:13 2023: cdo -O -s -f nc -remapc...
    CDO:                    Climate Data Operators version 2.1.1 (https://mpi...

In [4]:

import matplotlib.pyplot as plt

#fig, axs = plt.subplots(2,1)
#rfield[var][0,:,:].plot(ax=axs[0])
#cdofield[var][0,:,:].plot(ax=axs[1])


f = np.allclose(cdofield[var][0,:,:].data, rfield[var][0,:,:].data, equal_nan=True)
print(f.compute())
g = cdofield[var][0,:,:].data - rfield[var][0,:,:].data
#plt.hist(g)
#rfield.to_netcdf('alfa.nc')
#cdofield.to_netcdf('cdo.nc')


True


Test the different weights generation possibilities with CDO, tested with conservative remapping: the climtas code is way more efficient if files are already on the disk, since the call to CDO has to be done from file. CDO bindings have a minimum overhead to be considered

In [7]:
# nrepetition for the check
nr = 3

indir='tests/data'
filelist = ['tas-ecearth.nc', '2t-era5.nc', 'tos-fesom.nc']
tfile = os.path.join(indir, 'r360x180.nc')

# generate weights from file
for filein in filelist: 
 
    # open file
    xfield = xr.open_mfdataset(os.path.join(indir,filein))
    tfield = xr.open_mfdataset(tfile)

    # generate weights from file
    one = timeit.timeit(lambda: cdo_generate_weights(os.path.join(indir,filein), tfile, method = 'con'), number = nr)
    print(filein + ': Exectime climtas from file ' + str(one/nr))
    # generate weights from xarray
    two = timeit.timeit(lambda: cdo_generate_weights(xfield, tfield, method = 'con'), number = nr)
    print(filein + ': Exectime climtas from xarray ' + str(two/nr))
    # generatre weights with CDO bindings (from file)
    three = timeit.timeit(lambda: cdo.gencon(tfile, input = os.path.join(indir,filein), returnXDataset = True), number = nr)
    print(filein + ': Exectime cdo from file ' + str(three/nr))


tas-ecearth.nc: Exectime climtas from file 1.134233085360999
tas-ecearth.nc: Exectime climtas from xarray 1.3822929859937478
tas-ecearth.nc: Exectime cdo from file 1.10794160165824
2t-era5.nc: Exectime climtas from file 0.5474777166576436
2t-era5.nc: Exectime climtas from xarray 0.6316390913367892
2t-era5.nc: Exectime cdo from file 0.6012353887005398
tos-fesom.nc: Exectime climtas from file 14.036664524697699
tos-fesom.nc: Exectime climtas from xarray 20.836219373663578
tos-fesom.nc: Exectime cdo from file 13.07146831532009


In [10]:
# nrepetition for the check
nr = 3

indir='tests/data'
filelist = ['tas-ecearth.nc', '2t-era5.nc', 'tos-fesom.nc']
tfile = os.path.join(indir, 'r360x180.nc')

def smm_remap(ifile, tfile):

    xfield = xr.open_mfdataset(ifile)
    wfield = cdo_generate_weights(ifile, tfile, method = 'con')
    interpolator = Regridder(weights=wfield)
    var = list(xfield.data_vars)[-1]
    rfield = interpolator.regrid(xfield)
    return(rfield)

for filein in filelist: 

    one = timeit.timeit(lambda: cdo.remapcon(tfile, input = os.path.join(indir,filein), returnXDataset = True), number = nr)
    print(filein + ': Exectime CDO Weight+Remap ' + str(one/nr))
    two = timeit.timeit(lambda: smm_remap(os.path.join(indir,filein), tfile), number = nr)
    print(filein + ': Exectime SMM Weight+Remap ' + str(two/nr))


tas-ecearth.nc: Exectime CDO Weight+Remap 1.0530922710119437
tas-ecearth.nc: Exectime SMM Weight+Remap 1.1167515396373346
2t-era5.nc: Exectime CDO Weight+Remap 0.5952748576722419
2t-era5.nc: Exectime SMM Weight+Remap 0.5838990543658534
tos-fesom.nc: Exectime CDO Weight+Remap 12.258554262652373
tos-fesom.nc: Exectime SMM Weight+Remap 14.25126216133746


In [12]:
# nrepetition for the check
nr = 3

indir='tests/data'
filelist = ['tas-ecearth.nc', '2t-era5.nc', 'tos-fesom.nc']
tfile = os.path.join(indir, 'r360x180.nc')

for filein in filelist: 

    # CDO
    wfile = cdo.gencon(tfile, input = os.path.join(indir,filein))
    one = timeit.timeit(lambda: cdo.remap(tfile + ',' + wfile, input = os.path.join(indir,filein), returnXDataset = True), number = nr)
    print(filein + ': Exectime CDO Remap ' + str(one/nr))

    # SMM
    xfield = xr.open_mfdataset(os.path.join(indir,filein))
    wfield = cdo_generate_weights(os.path.join(indir,filein), tfile, method = 'con')
    interpolator = Regridder(weights=wfield)
    two = timeit.timeit(lambda: interpolator.regrid(xfield), number = nr)
    print(filein + ': Exectime SMM Remap ' + str(two/nr))

tas-ecearth.nc: Exectime CDO Remap 0.2639976349892095


SystemExit: Cannot process this source_data, sure it is xarray?

/work/users/paolo/miniconda3/envs/DevECmean4/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
